# setup

In [ ]:
!pip install ultralytics

In [ ]:
!ls -l

In [ ]:
import os, yaml, shutil, cv2, time
import numpy as np
from ultralytics import YOLO
from IPython.display import FileLink

In [ ]:
def create_yaml_file(dataset_path):
    base_dir = os.path.abspath(dataset_path)
    data_config = {
        "train": os.path.join(base_dir, "train/images"),
        "val": os.path.join(base_dir, "val/images"),
        "train_labels": os.path.join(base_dir, "train/labels"),
        "val_labels": os.path.join(base_dir, "val/labels"),
        "nc": 1,
        "names": ["polyp"]
    }
    yaml_path = "/kaggle/working/data-" + os.path.basename(dataset_path) + ".yaml"
    with open(yaml_path, "w") as f:
        yaml.dump(data_config, f)
    return yaml_path

yaml_mini_path = create_yaml_file("/kaggle/input/sun-seg-mini")  # 4K train, 1K val
yaml_30k_path = create_yaml_file("/kaggle/input/sun-seg-30k-processed")  # 20K train, 5K val

# train models

## yolo8

### nano

In [ ]:
model = YOLO("/kaggle/working/yolov8n-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolov8n-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

### small

In [ ]:
model = YOLO("/kaggle/working/yolov8s-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolov8s-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

### medium

In [ ]:
model = YOLO("/kaggle/working/yolov8m-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolov8m-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

### large

In [ ]:
model = YOLO("/kaggle/working/yolov8l-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolov8l-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

### extra large

In [ ]:
model = YOLO("/kaggle/working/yolov8x-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolov8x-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

## yolo11

### nano

In [ ]:
model = YOLO("/kaggle/working/yolo11n-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolo11n-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

### small

In [ ]:
model = YOLO("/kaggle/working/yolo11s-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolo11s-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

### medium

In [ ]:
model = YOLO("/kaggle/working/yolo11m-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolo11m-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

### large

In [ ]:
model = YOLO("/kaggle/working/yolo11l-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolo11l-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

### extra large

In [ ]:
model = YOLO("/kaggle/working/yolo11x-seg.pt")
model.train(data=yaml_mini_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

In [ ]:
model = YOLO("/kaggle/working/yolo11x-seg.pt")
model.train(data=yaml_30k_path, batch=8, epochs=50, imgsz=512, patience=5, augment=False)

## download results

In [ ]:
!zip -r yolov8n_datamini.zip /kaggle/working/runs/segment/train22

In [ ]:
FileLink("yolov8n_datamini.zip")

# test models

In [ ]:
def evaluate_model(model_path, image_folder, mask_folder):

    model = YOLO(model_path, verbose=False)

    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))])
    mask_files = sorted([f for f in os.listdir(mask_folder) if f.endswith(('.jpg', '.png', '.jpeg'))])

    total_iou = 0
    total_time = 0
    tp = 0
    fp = 0
    fn = 0
    count = 0

    for img_file, mask_file in zip(image_files, mask_files):

        img_path = os.path.join(image_folder, img_file)
        image = cv2.imread(img_path)

        start_time = time.time()
        results = model(image, verbose=False)
        end_time = time.time()
        total_time += (end_time - start_time)

        mask_path = os.path.join(mask_folder, mask_file)
        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) > 128

        if not results or not results[0].masks:
            if gt_mask.any():
                fn += 1
            continue

        pred_mask = results[0].masks.data.cpu().numpy()[0] > 0.5

        intersection = np.logical_and(pred_mask, gt_mask).sum()
        union = np.logical_or(pred_mask, gt_mask).sum()
        iou = intersection / union if union > 0 else 0

        total_iou += iou
        count += 1

        if gt_mask.any() and pred_mask.any():
            tp += 1
        elif gt_mask.any() and not pred_mask.any():
            fn += 1
        elif not gt_mask.any() and pred_mask.any():
            fp += 1

    avg_iou = total_iou / count if count > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    avg_fps = count / total_time if total_time > 0 else 0

    print(f"IoU: {avg_iou*100:.1f}% | Dice (F1): {f1_score*100:.1f}% | Recall: {recall*100:.1f}% | FPS: {int(avg_fps)}")
    return avg_iou, f1_score, recall, avg_fps

In [ ]:
def test_model_on_datasets(model_path, test_datasets):
    for test_dataset in test_datasets:
        print(f"\ntest dataset: {test_dataset['name']}")
        evaluate_model(model_path, test_dataset["images"], test_dataset["masks"])

In [ ]:
F_TEST_SUNSEG_POS = {
    "name": "SUN-SEG positive",
    "images": "/kaggle/input/sun-seg-mini/test/images/positive",
    "masks": "/kaggle/input/sun-seg-mini/test/masks/positive"
}  # 1500 frames
F_TEST_SUNSEG_NEG = {
    "name": "SUN-SEG negative",
    "images": "/kaggle/input/sun-seg-mini/test/images/negative",
    "masks": "/kaggle/input/sun-seg-mini/test/masks/negative"
}  # 500 frames
F_TEST_ETISLARIB = {
    "name": "ETIS-Larib",
    "images": "/kaggle/input/etis-larib-cropped-scaled/images",
    "masks": "/kaggle/input/etis-larib-cropped-scaled/masks",
}  # 196 frames
F_TEST_KSAVIR = {
    "name": "Ksavir-SEG",
    "images": "/kaggle/input/ksavir-seg-cropped-scaled/images",
    "masks": "/kaggle/input/ksavir-seg-cropped-scaled/masks"
}  # 1000 frames

TEST_DATASETS = [F_TEST_SUNSEG_POS, F_TEST_SUNSEG_NEG, F_TEST_ETISLARIB, F_TEST_KSAVIR]

In [ ]:
test_model_on_datasets("/kaggle/path/to/model.pt", TEST_DATASETS)